In [1]:
pip install numpy pandas seaborn matplotlib plotly SQLAlchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import related libaries
import numpy as np
import pandas as pd
import os
from sqlalchemy import create_engine
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit



In [4]:

spark = SparkSession.builder \
    .appName("PySpark PostgreSQL and Cassandra Example") \
    .master("local[*]") \
    .config("spark.executor.memory", "2g") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.2,com.datastax.spark:spark-cassandra-connector_2.12:3.0.0") \
    .config("spark.cassandra.connection.host", "cassandra") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.cassandra.auth.username", "admin") \
    .config("spark.cassandra.auth.password", "admin") \
    .getOrCreate()

In [8]:
df_postgres = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/demo") \
    .option("dbtable", "bos_air_traffic") \
    .option("user", "postgres") \
    .option("password", "1234qwer") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_postgres.show()



+------+--------+--------------+-------------+------------+---------+--------+-------------+---------+--------+----------+-------------+-------+------------+------+-----+---------------+
|icao24|callsign|origin_country|time_position|last_contact|longitude|latitude|baro_altitude|on_ground|velocity|true_track|vertical_rate|sensors|geo_altitude|squawk|  spi|position_source|
+------+--------+--------------+-------------+------------+---------+--------+-------------+---------+--------+----------+-------------+-------+------------+------+-----+---------------+
|4b1881|SWR52   |   Switzerland|   1713398182|  1713398182| -71.0071| 42.3607|       -68.58|    false|   70.63|     20.02|        -0.65|   NULL|      -15.24|  NULL|false|              0|
|a0311d|DAL2420 | United States|   1713398182|  1713398182| -71.0167| 42.3573|         NULL|     true|    2.83|    236.25|         NULL|   NULL|        NULL|  NULL|false|              0|
|49530a|TAP215  |      Portugal|   1713397890|  1713397890| -71.0

In [11]:
# Read CSV files from a directory
df_csv = spark.read \
    .format("csv") \
    .option("header", True) \
    .option("inferSchema", "true") \
    .load("./sensorInput")  

df_csv.show()

+------+--------+--------------------+-------------+------------+---------+--------+-------------+---------+--------+----------+-------------+-------+------------+------+-----+---------------+
|icao24|callsign|      origin_country|time_position|last_contact|longitude|latitude|baro_altitude|on_ground|velocity|true_track|vertical_rate|sensors|geo_altitude|squawk|  spi|position_source|
+------+--------+--------------------+-------------+------------+---------+--------+-------------+---------+--------+----------+-------------+-------+------------+------+-----+---------------+
|a6cacb|DAL408  |       United States|   1713400742|  1713400742| -71.0066|  42.378|         NULL|     true|    2.31|     19.69|         NULL|   NULL|        NULL|  NULL|false|              0|
|a35d73|DAL1513 |       United States|   1713400742|  1713400742| -71.0235| 42.3614|         NULL|     true|    2.83|     180.0|         NULL|   NULL|        NULL|  NULL|false|              0|
|a22ac6|RPA5622 |       United Stat

In [14]:

from pyspark.sql.functions import col, row_number, length
from pyspark.sql.window import Window
from pyspark.sql.functions import lit

# Continuing from the previous step where df_csv was loaded


# Union the data from PostgreSQL and CSV (adjust the renaming to match your schema)
df_combined = df_postgres.unionByName(df_csv)

# Define the window specification for deduplication based on some criteria like 'callsign'
windowSpec = Window.partitionBy("callsign").orderBy(col("last_contact").desc())

# Apply the window function to add a row number within each partition
df_combined_with_row_number = df_combined.withColumn("row_num", row_number().over(windowSpec))

# Filter to get only the latest records for each 'callsign', assuming 'callsign' must be non-empty
df_latest = df_combined_with_row_number.filter(
    (col("row_num") == 1) &
    col("callsign").isNotNull() &
    (length(col("callsign")) > 0)
).select(
    "callsign", "longitude", "latitude", "on_ground", "squawk"
)

# Show the DataFrame to verify the content
df_latest.show()




+--------+---------+--------+---------+------+
|callsign|longitude|latitude|on_ground|squawk|
+--------+---------+--------+---------+------+
|AAL1074 | -71.0202| 42.3614|     true|6246.0|
|AAL2148 | -71.0186| 42.3603|     true|  NULL|
|ASA311  | -71.0172| 42.3636|     true|  NULL|
|DAL1513 | -71.0057| 42.3784|     true|  NULL|
|DAL154  | -71.0245| 42.3722|     true|  NULL|
|DAL1707 | -71.0245| 42.3639|     true|5751.0|
|DAL174  | -70.8058| 42.3828|    false|7137.0|
|DAL2420 | -71.0167| 42.3573|     true|  NULL|
|DAL262  | -71.1536| 42.3763|    false|1047.0|
|DAL385  | -70.9707| 42.3635|    false|  NULL|
|DAL408  | -70.7663| 42.3443|    false|  NULL|
|DAL937  | -71.0023| 42.3746|     true|2070.0|
|EJA377  | -71.0217| 42.3742|     true|2234.0|
|EJA628  | -71.0207| 42.3739|     true|6313.0|
|EJA918  |  -71.022| 42.3741|     true|  NULL|
|FDX3714 | -70.9791| 42.3796|    false|4075.0|
|IBE2625 | -71.0173| 42.3699|     true|  NULL|
|ICE630  | -71.0172| 42.3529|    false|  NULL|
|JBU1246 | -7

In [15]:
# Write the results back to Cassandra in a table designed for latest flight data
df_latest.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode('append') \
    .option("keyspace", "demo") \
    .option("table", "latest_flight_data") \
    .save()